In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from pandas.plotting import scatter_matrix

import statsmodels.formula.api as smf

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 200

In [2]:
def load_lending_club(path_):
    date_cols = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
    return pd.read_csv(path_, 
                 low_memory=False, 
                 parse_dates=date_cols, 
                 date_format="%b-%Y")

In [12]:
def tweak_lending_club(df_, ohe=False):
    drop_cols = ["debt_settlement_flag", "Unnamed: 0",  
                 "last_fico_range_low", "num_bc_sats", "num_bc_tl", 
                 "fico_range_low", "num_rev_accts", "out_prncp", 
                 "out_prncp_inv", "out_prncp_inv", "policy_code", 
                 "funded_amnt_inv", "loan_amnt", 
                 "collection_recovery_fee", "last_credit_pull_d", "id", 
                 "url", "pymnt_plan", "emp_title", 
                 "title", "zip_code", "verification_status", "last_pymnt_d"]
    high_nan_cols = ['all_util', 'annual_inc_joint', 'deferral_term', 
                     'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 
                     'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 
                     'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 
                     'hardship_status', 'hardship_type', 'il_util', 
                     'inq_fi', 'inq_last_12m', 'max_bal_bc', 
                     'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 
                     'mths_since_rcnt_il', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 
                     'next_pymnt_d', 'open_acc_6m', 'open_act_il', 
                     'open_il_12m', 'open_il_24m', 'open_rv_12m', 
                     'open_rv_24m', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 
                     'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 
                     'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 
                     'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 
                     'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_revol_util', 
                     'total_bal_il', 'total_cu_tl', 'verification_status_joint']
    transformed_cols = ["term", "initial_list_status", 
                        "application_type", "hardship_flag", "emp_length", 
                        "int_rate", "revol_util"]
    object_cols = ["grade", "sub_grade", "home_ownership", 
                   "purpose", "addr_state", "issue_date_month"]
    
    
    LC_df_ = df_.assign(target = np.where((df_.loan_status == "Charged Off") | 
                                        (df_.loan_status == "Does not meet the credit policy. Status:Charged Off") | 
                                        (df_.loan_status == "Default"), 
                                        1, 0), 
                      term_36months = np.where(df_.term == "60 months", 0, 1), 
                      initial_list_status_w = np.where(df_.initial_list_status == "f", 0, 1), 
                      individual = np.where(df_.application_type == "Joint App", 0, 1),
                      hardhship = np.where(df_.hardship_flag == "Y", 1, 0),
                      employed_over_10yrs = np.where(df_.emp_length == "10+ years", 1, 0),
                      emp_years = df_.emp_length.map({"< 1 year": 0, "1 year": 1, "2 years": 2, 
                                                        "3 years": 3, "4 years": 4, "5 years": 5, 
                                                        "6 years": 6, "7 years": 7, "8 years": 8, 
                                                        "9 years": 9, "10+ years": 10}),
                      interest_rate = df_.int_rate.str.replace('%', '').astype(float),
                      revolving_util = df_.revol_util.str.replace('%', '').astype(float),
                      issue_date_month = df_.issue_d.dt.month.astype(str),
                      issue_date_year = df_.issue_d.dt.year,
                      years_since_earliest_cr_line = df_.issue_d.dt.year - df_.earliest_cr_line.dt.year,
                     ).drop(labels=drop_cols+high_nan_cols+transformed_cols, axis=1)
    
    print("Note: \"target\" series generated from \"loan_status\" series.\"loan_status\" not dropped.")
        
    if ohe:
        return(pd.get_dummies(LC_df_, columns=object_cols).fillna(method="ffill"))
    else:
        return LC_df_
    

In [50]:
definitions = pd.read_csv("LC_definitions.csv")

def print_def(variable_name):
    """
    Pass in a variable name as a string, print its defintion.
    Definitions included with the original dataset. 
    Only works with ORIGINAL feature names.
    """
    print(variable_name, "-", definitions.loc[definitions.LoanStatNew == variable_name, "Description"].iloc[0])

print_def("addr_state")

addr_state - The state provided by the borrower in the loan application


In [42]:
# All features in current dataset
for item in df.columns:
    try:
        print_def(item)
    except:
        print(f"****{item} - new feature.")

funded_amnt - The total amount committed to that loan at that point in time.
installment - The monthly payment owed by the borrower if the loan originates.
grade - LC assigned loan grade
sub_grade - LC assigned loan subgrade
home_ownership - The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc - The self-reported annual income provided by the borrower during registration.
issue_d - The month which the loan was funded
loan_status - Current status of the loan
purpose - A category provided by the borrower for the loan request. 
addr_state - The state provided by the borrower in the loan application
dti - A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
delinq_2yrs - The number of 30+ days past-due incidences of delinquency in the bor

In [46]:
df = tweak_lending_club(load_lending_club("lending_club_sample.csv"))
print(df.shape)

Note: "target" series generated from "loan_status" series."loan_status" not dropped.
(300000, 76)


## Quick and dirty biased sample
Heavily samples people with low emp_years that did not default/charge-off

In [48]:
def biased_sample(df_):
    """
    Heavily samples from low-employment experience w/o default.
    Returns feature matrix and target vector.
    """
    biased_df = df_.sort_values(by = ["target", "emp_years"]).loc[:2000, :].sample(1000, replace=False)
    
    biased_df = pd.concat([df.sample(1000), biased_df])
    
    return biased_df.drop(labels="target", axis=1), biased_df["target"]

In [49]:
biased_sample(df).to_csv("lending_club_biased_preprocessed.csv", index=False)

In [ ]:
X_bias, y_bias = biased_sample(df)

# Useful column drops 

if dummied using pd.get_dummies( ) at import

In [ ]:
# The last item of each categorical dummy set and the datetime series.
logit_drop_cols = ["grade_G", "sub_grade_G5", "home_ownership_NONE", 
                   "purpose_other", "addr_state_WY", "issue_date_month_12"] + list(df.select_dtypes(include='datetime').columns)

logit_drop_cols_grade = ["grade_A", "grade_B", "grade_C", 
                         "grade_D", "grade_E","grade_F"]

logit_drop_cols_subgrade = ["sub_grade_A1", "sub_grade_A2", "sub_grade_A3", 
                             "sub_grade_A4", "sub_grade_A5", "sub_grade_B1", 
                             "sub_grade_B2", "sub_grade_B3", "sub_grade_B4", 
                             "sub_grade_B5", "sub_grade_C1", "sub_grade_C2", 
                             "sub_grade_C3", "sub_grade_C4", "sub_grade_C5", 
                             "sub_grade_D1", "sub_grade_D2", "sub_grade_D3", 
                             "sub_grade_D4", "sub_grade_D5", "sub_grade_E1", 
                             "sub_grade_E2", "sub_grade_E3", "sub_grade_E4", 
                             "sub_grade_E5", "sub_grade_F1", "sub_grade_F2", 
                             "sub_grade_F3", "sub_grade_F4", "sub_grade_F5", 
                             "sub_grade_G1", "sub_grade_G2", "sub_grade_G3", 
                             "sub_grade_G4"]

logit_state_dummies = ["addr_state_AK", "addr_state_AL", "addr_state_AR", 
                 "addr_state_AZ", "addr_state_CA", "addr_state_CO", 
                 "addr_state_CT", "addr_state_DC", "addr_state_DE", 
                 "addr_state_FL", "addr_state_GA", "addr_state_HI", 
                 "addr_state_ID", "addr_state_IL", "addr_state_IN", 
                 "addr_state_KS", "addr_state_KY", "addr_state_LA", 
                 "addr_state_MA", "addr_state_MD", "addr_state_ME", 
                 "addr_state_MI", "addr_state_MN", "addr_state_MO", 
                 "addr_state_MS", "addr_state_MT", "addr_state_NC", 
                 "addr_state_ND", "addr_state_NE", "addr_state_NH", 
                 "addr_state_NJ", "addr_state_NM", "addr_state_NV", 
                 "addr_state_NY", "addr_state_OH", "addr_state_OK", 
                 "addr_state_OR", "addr_state_PA", "addr_state_RI", 
                 "addr_state_SC", "addr_state_SD", "addr_state_TN", 
                 "addr_state_TX", "addr_state_UT", "addr_state_VA", 
                 "addr_state_VT", "addr_state_WA", "addr_state_WI", 
                 "addr_state_WV"]

logit_issue_date_month_dummies = ["issue_date_month_1", "issue_date_month_10", "issue_date_month_11", 
                            "issue_date_month_2", "issue_date_month_3", 
                            "issue_date_month_4", "issue_date_month_5", "issue_date_month_6", 
                            "issue_date_month_7", "issue_date_month_8", "issue_date_month_9"] 

logit_drop_all = logit_drop_cols + logit_drop_cols_subgrade + logit_state_dummies + logit_issue_date_month_dummies

In [43]:
# Either pass in a list of columns to use via logit_cols_ 
# or pass in a list of columns to drop via logit_drop_
def smf_logit(df_, y_, logit_cols_=None, logit_drop_=None):
    if logit_drop_:
        sm_df = df_.drop(labels=logit_drop_, axis=1).dropna()
    elif logit_cols_:
        sm_df = df_.loc[:, logit_cols_+[y_]].copy().dropna()   
    else:
        sm_df = df_.copy().dropna()
        
    formula = y_ + " ~ "
    for column in sm_df.drop(labels=[y_], axis=1).columns:
        formula += column + " + "
    formula = formula[:-3]
    
    print(f"{formula}")
    sm_model = smf.logit(formula, data=sm_df).fit()
    print(sm_model.summary())

In [44]:
use_cols = ["last_fico_range_high", "last_pymnt_amnt", "total_rec_prncp", 
            "funded_amnt", "issue_date_year", "installment", 
            "total_pymnt_inv", "total_pymnt"]

smf_logit(df, "target", logit_cols_=use_cols)

target ~ last_fico_range_high + last_pymnt_amnt + total_rec_prncp + funded_amnt + issue_date_year + installment + total_pymnt_inv + total_pymnt
Optimization terminated successfully.
         Current function value: 0.110045
         Iterations 13
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:               300000
Model:                          Logit   Df Residuals:                   299991
Method:                           MLE   Df Model:                            8
Date:                Tue, 08 Aug 2023   Pseudo R-squ.:                  0.7081
Time:                        21:17:51   Log-Likelihood:                -33013.
converged:                       True   LL-Null:                   -1.1310e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------